# **Le classique "chien ou chat" - CNN avec Keras**

Le but de cet exercice est la classification d'images d'animaux en chien ou en chat. Évidemment ça peut paraître assez stérile comme objectif dans la vraie vie mais sachez que les applications intéressantes et utiles de ces réseaux sont multiples notamment en médecine.  
Par exemple pour la reconnaissance de tumeur pulmonaire ou autre.  
Et ce qui est chouette, c'est que la seule différence sera dans les images pour l'entraînement et la strutucre du réseau éventuellement.

## **Les données**

Les données ne sont plus structurées en un seul et joli CSV à partir duquel on peut associer le label et les variables explicatives facilement pour chaque observation. Le problème est qu'il faut pouvoir associer à chaque image le bon label.

Une idée simple de solution est de structurer les données dans des dossiers (sous-dossiers) contenant les labels dans les noms par exemple :
1. séparer les échantillons d'entraînement et de test dans 2 dossiers différents
2. ensuite on appelle chaque image par un nom {label}_{numéro} genre chat_1, chat_2, chien_1, chien_2 etc...
3. pour finir on fait un petit script qui récupère le label dans le nom de l'image à chaque fois via une petite manip de string

On pourrait faire ça, ça marche, mais il y a une manière "toute cousue" qui fonctionne très bien dans Keras puisqu'on on y trouve des outils pour extraire les images facilement. La seule contrainte est de bien structurer le dossier de la manière suivante :

<img src="images/cnn_exo_arborescence.png">

## **Le réseau CNN**

Tout est dit. Ou presque.  
Vous devez construire et entraîner un réseau de neurones à convolution pour la catégorisation des images en chien/chat.  
Allez, je vous mets quelques pistes et même toutes les librairies nécessaires, c'est cadeau.

In [1]:
# Librairies
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dense
from keras.preprocessing.image import ImageDataGenerator

2024-01-17 15:57:13.948311: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### **L'architecture**

Dans un premier temps construisez votre réseau étape par étape

In [2]:
# 0. Initialisation
classifier = Sequential()

# 1. La Convolution
classifier.add(Conv2D(filters=32, kernel_size=(3, 3),
                      input_shape = (64, 64, 3), activation = 'relu'))

# 2. Le Max Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Ajout d'une couche de convolution/pooling supplémentaire
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# 3. Le Flattening
classifier.add(Flatten())

# 4. Les couches fully-connected
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Paramètres d'entraînement
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

2024-01-17 15:57:20.226523: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

#### **L'entraînement**

Là, c'est un peu plus complexe alors je vous donne les méthodes à utiliser, dans cet ordre :
1. `ImageDataGenerator`
2. `flow_from_directory`
3. `fit`

Il va certainement vous falloir fouiller un peu là-dedans https://keras.io/api/preprocessing/image/ et en profiter pour découvrir le concept merveilleux de *Data Augmentation*.

In [4]:
# On génère de nouvelles images de training supplémentaires avec ImageDataGenerator
# qui au passage gère le preprocessing de l'image (notamme le rescaling)
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit(training_set,
               epochs = 25,
               validation_data = test_set)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
250/250 [==============================] - 42s 167ms/step - loss: 0.6552 - accuracy: 0.6070 - val_loss: 0.5974 - val_accuracy: 0.6775
Epoch 2/25
250/250 [==============================] - 43s 171ms/step - loss: 0.5907 - accuracy: 0.6839 - val_loss: 0.5424 - val_accuracy: 0.7350
Epoch 3/25
250/250 [==============================] - 42s 168ms/step - loss: 0.5469 - accuracy: 0.7210 - val_loss: 0.5014 - val_accuracy: 0.7675
Epoch 4/25
250/250 [==============================] - 42s 169ms/step - loss: 0.5143 - accuracy: 0.7484 - val_loss: 0.5719 - val_accuracy: 0.7255
Epoch 5/25
250/250 [==============================] - 41s 165ms/step - loss: 0.5022 - accuracy: 0.7561 - val_loss: 0.5111 - val_accuracy: 0.7545
Epoch 6/25
250/250 [==============================] - 42s 166ms/step - loss: 0.4693 - accuracy: 0.7720 - val_loss: 0.4740 - val_accuracy: 0.7870
Epoch 7/25
250/250 [==========================

## **Prédictions à partir de nouvelles images**

Prenez une jolie photo de votre chien ou de votre chat et utilisez votre modèle CNN pour prédire de quel animal il s'agit. Si vous n'avez ni chien ni chat, pas de panique, dans le dossier `single_prediction`, il y a ce qu'il faut et vous pouvez aussi tester d'autres images en allant en récupérer sur internet.

## **Amélioration de votre modèle**

Un petit *kaggle* maison : essayez d'améliorer l'accuracy de votre CNN
- Accuracy **entre 80% et 85%** sur le jeu de test, c'est **pas mal**
- Accuracy **entre 85% et 90%** sur le jeu de test, c'est **bien**
- Accuracy **entre 90% et 95%**  sur le jeu de test, c'est **très bien**
- Accuracy **supérieur à 95%**  sur le jeu de test, c'est **top**

Les règles du jeu :
- On garde bien sûr le même jeu d'entraînement
- Il est interdit de spécifier manuellement une seed

*Une indication utile :* laissez tomber la validation croisée dans un premier temps car en utilisant `fit_generator`, le modèle passe directement sur l'échantillon test pour valider le modèle. C'est faisable mais un peu plus compliqué...

```python
# éventuellement ajout d'époques pour voir si on améliore encore ou si ça converge avant
classifier.fit(training_set,
               epochs = 40,
               validation_data = test_set)
```

Pour aller plus loin, on peut modifier le réseau et atteindre les 90% d'accuracy espérés de différentes manières en rendant le réseau un peu plus complexe : vous pouvez notamment ajouter une couche de convolution ou augmenter le nombre de neurones dans les couches. Le problème c'est que c'est un peu long de réentraîner à chaque fois tout le modèle pour tester les améliorations.

#### **Autre solution: le transfer learning**

In [5]:
# Voilà à quoi ressemble le modèle VGG16 full
from keras.applications.vgg16 import VGG16
vgg16 = VGG16(weights='imagenet')
print(vgg16.summary())

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     